In [1]:
import math
import time
import threading
import numpy as np

from autofunc import AutoFunc
from python_api_class import PythonAPI
from concurrent.futures import ThreadPoolExecutor

from enum import Enum 

class MoveDirection(Enum): 
    Diagonal =      0
    Vertical =      1
    Horizontal =    2

In [2]:
api_handle = PythonAPI()
af = AutoFunc(api_handle)

In [3]:
# api_handle.send_command(f"sim")
api_handle.send_command("no_detect add {} {} {} {}".format(35, 35, 1030, 670)) 

# Constants
ONE_PIXEL_LEN = 0.112  # Length in reality corresponding to one pixel in BPU_Tool
DROPLET_HEIGHT = 0.2   # Height of the droplet
COEFFICIENT = math.pow(ONE_PIXEL_LEN, 2) * DROPLET_HEIGHT  # Coefficient for volume calculation
ONE_UL_LEN = 19.96489265625  # Edge length of 1 ul droplet on UI (assuming a square droplet)

START_x1, START_y1 = 180, 260
START_x2, START_y2 = 390, 260
START_x3, START_y3 = 600, 260
START_x4, START_y4 = 810, 260

list_start_x = list(range(180, 1020, 105))
list_start_y = [75, 400]

In [4]:
api_handle.send_command(f"vbox add {1072 - 894} {409.5} {1072 - 992} {634.5}")

-1

In [5]:
mag_centers = af.get_mag_info(8, 4)

In [6]:

y_offset = 35

def get_mags(begin_row, begin_col, end_row, end_col, arrange=0):
    # 先定义整个32个点的矩阵
    global mag_centers
    matrix = mag_centers
    
    # 定义行列数
    rows, cols = 8, 4
    
    # 提取指定范围内的子矩阵
    sub_matrix = []
    
    if arrange == 0:  # 列优先
        for c in range(begin_col, end_col + 1):
            for r in range(begin_row, end_row + 1):
                index = c * rows + r
                sub_matrix.append(matrix[index])
    else:  # 行优先
        for r in range(begin_row, end_row + 1):
            for c in range(begin_col, end_col + 1):
                index = c * rows + r
                sub_matrix.append(matrix[index])
    
    return sub_matrix

def move_to_x(drp_id, x, wait = False):
    _, y1, _, y2 = api_handle.send_command(f"getbox {drp_id}")
    y = (y1 + y2) / 2
    tag = api_handle.send_command(f"move_p {drp_id} {x} {y} {MoveDirection.Horizontal.value}")
    if wait:
        api_handle.wait_tag(tag)
    return tag
    
def move_to_y(drp_id, y, wait = False):
    x1, _, x2, _ = api_handle.send_command(f"getbox {drp_id}")
    x = (x1 + x2) / 2
    tag = api_handle.send_command(f"move_p {drp_id} {x} {y} {MoveDirection.Vertical.value}")
    if wait:
        api_handle.wait_tag(tag)
    return tag

def get_4drps_list_vertical(begin, end):
    # 生成从 begin 到 end 的倒序列表
    num_list = list(range(end, begin - 1, -1))
    
    # 将数字分为两部分，分别为每列的数据
    col1 = num_list[::2]  # 选择奇数索引
    col2 = num_list[1::2]  # 选择偶数索引
    
    # 创建结果列表
    result = []
    
    # 每次处理4个元素
    for i in range(0, len(col1), 4):
        # 添加列1的4个元素
        result.append(col1[i:i+4])
        # 添加列2的4个元素
        result.append(col2[i:i+4])
    
    return result

def group_list(input_list):
    odd_index_elements = input_list[0::2]  # 选取奇数项索引的元素
    even_index_elements = input_list[1::2]  # 选取偶数项索引的元素
    return odd_index_elements, even_index_elements 

def split_into_groups(input_list, num_groups):
    # 计算每组的大小
    group_size = len(input_list) // num_groups
    # 创建组列表
    groups = [input_list[i*group_size:(i+1)*group_size] for i in range(num_groups)]
    # 将剩余的元素加入到最后一组（如果有多余的元素）
    remaining_elements = len(input_list) % num_groups
    if remaining_elements:
        groups[-1].extend(input_list[-remaining_elements:])
    
    return groups

def get_droplet_central_point(uid):
    uid = int(uid)  # 提前转换uid为整数
    x1, y1, x2, y2 = api_handle.send_command(f"getbox {uid}")
    # 计算中心点坐标并返回
    return (x1 + x2) / 2, (y1 + y2) / 2

# Convert a list of numbers to a space-separated string
def list_to_string(numbers):
    return ' '.join(map(str, numbers))

def get_area_central_point(area): 
    if (4 != len(area)) :
        return -1, -1
    return (area[0] + area[2]) / 2, (area[1] + area[3]) / 2

def process_list(input_list):
    # 使用列表解构和reversed，避免中间变量和原地修改
    g1, g2 = group_list(input_list)
    return list(reversed(g1)) + list(reversed(g2))

def move_drps_to_tmp_pos(drps_list: list, dest_mags: list, dir: str):
    tmp_tags = []
    dir_stringlist = ['up', 'right', 'left', 'right']
    sign = -1 if (dir == dir_stringlist[0] or dir == dir_stringlist[3]) else 1
    for drp_id, pos in zip(drps_list, dest_mags):
        # 将液滴摆放在目标磁控点的上下方向
        if (dir in dir_stringlist[:2]):
            tag = api_handle.send_command(f"move_p {drp_id} {pos[0]} {pos[1] + sign * y_offset} {MoveDirection.Vertical.value}")
            tmp_tags.append(tag)
        # 左右的逻辑还没想好
        else:
            pass
    af.wait_tag_all(tmp_tags)

def move_drps_to_points_v(drps: list, positions: list):
    tags = []
    for id, pos in zip(drps, positions):
        tag = api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Vertical.value}")
        tags.append(tag)
    af.wait_tag_all(tags)
    
def move_drps_to_points_h(drps: list, positions: list):
    tags = []
    for id, pos in zip(drps, positions):
        tag = api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Horizontal.value}")
        tags.append(tag)
    af.wait_tag_all(tags)

In [7]:
def row_to_column_sort(data, rows = 6, cols = 4):
    """
    将一个按行排序的列表转换为按列排序
    
    参数:
    data -- 输入列表，按行排序的数据
    
    返回:
    列表，按列排序后的数据
    """
    # 确保输入是列表
    if not isinstance(data, list):
        raise TypeError("输入必须是列表")
    
    # 将输入列表转换为NumPy数组，假设是6×4的形状
    arr = np.array(data).reshape(rows, cols)
    
    # 将数组按列展开，即先取所有行的第0列，然后是第1列，依此类推
    column_ordered = arr.T.flatten()
    
    # 转换回Python列表
    result = column_ordered.tolist()
    
    return result


In [8]:
def get_hole_pos(dir: str, idx: int) -> list:
    """
    获取芯片孔位坐标
    
    参数:
        dir (str): 方向 - 'l'左, 'r'右, 'u'上, 'd'下
        idx (int): 孔位索引，从1开始
        
    返回:
        list: [x坐标, y坐标]
    """
    # 定义各个方向的孔位坐标
    holes_config = {
        'r': {
            'x': 988,
            'y': [115, 225, 330, 435, 545, 620]
        },
        'l': {
            'x': 75,
            'y': [85, 175, 280, 385, 490, 595]
        },
        'd': {
            'x': [100, 190, 295, 400, 510, 615, 725, 830, 940],
            'y': 640
        },
        'u': {
            'x': [135, 240, 345, 455, 560, 670, 775, 885, 965],
            'y': 70
        }
    }
    
    # 检查参数有效性
    if dir not in holes_config:
        raise ValueError("方向参数必须是 'l', 'r', 'u' 或 'd'")
    
    holes = holes_config[dir]
    
    # 获取该方向的最大索引值
    max_idx = len(holes['y']) if isinstance(holes['x'], int) else len(holes['x'])
    
    if idx < 1 or idx > max_idx:
        raise ValueError(f"索引值必须在1到{max_idx}之间")
    
    # 将索引转换为0基数
    idx = idx - 1
    
    # 返回坐标
    if isinstance(holes['x'], int):
        # 左右方向：x坐标固定，y坐标是列表
        return [holes['x'], holes['y'][idx]]
    else:
        # 上下方向：x坐标是列表，y坐标固定
        return [holes['x'][idx], holes['y']]

def get_hole_pos_by_pipe_id(pipe_id: int):
    if (0 != pipe_id % 2):
        return [-1, -1]
    # 左侧孔位坐标
    if (pipe_id <= 10):
        return get_hole_pos('r', (10 - pipe_id) // 2 + 1)
    elif (pipe_id >= 12 and pipe_id <= 28):
        return get_hole_pos('u', (28 - pipe_id) // 2 + 1)
    elif (pipe_id >= 30 and pipe_id <= 40):
        return get_hole_pos('l', (pipe_id - 30) // 2 + 1)
    elif (pipe_id >= 42 and pipe_id <= 58):
        return get_hole_pos('d', (pipe_id - 42) // 2 + 1)

def wait_tag(tag: int):
    api_handle.wait_tag(tag)

In [9]:
def single_mix(drp_id, mix_time):
    x1, y1, x2, y2 = api_handle.send_command(f"getbox {drp_id}")
    api_handle.send_command(f"set_virtual {drp_id} {0}")
    move_length = 0.7
    list_direction = [(move_length*(-1),0), (-move_length*2**0.5/2,move_length*2**0.5/2), (0,move_length), (move_length*2**0.5/2,move_length*2**0.5/2), (move_length,0), (move_length*2**0.5/2,-move_length*2**0.5/2), (0,move_length*(-1)), (-move_length*2**0.5/2,-move_length*2**0.5/2), (move_length*(-1),0), (-move_length*2**0.5/2,-move_length*2**0.5/2), (0,move_length*(-1)), (move_length*2**0.5/2,-move_length*2**0.5/2), (move_length,0), (move_length*2**0.5/2,move_length*2**0.5/2), (0,move_length), (-move_length*2**0.5/2,move_length*2**0.5/2)]
    running_time = 0
    time_interval = 0.4
    while running_time <= mix_time:
        for direction in list_direction:
            sx_left = x1+(x2-x1)*direction[0]
            sx_right = sx_left + x2 - x1
            sy_right = y1+(y2-y1)*direction[1]
            sy_up = sy_right + y2 - y1
            vbox_id = api_handle.send_command(f"vbox add {sx_left} {sy_right} {sx_right} {sy_up}")
            time.sleep(time_interval)
            api_handle.send_command(f"vbox remove {vbox_id}")
            running_time += time_interval
    vbox_id = api_handle.send_command(f"vbox add {x1} {y1} {x2} {y2}")
    drp_id_new = api_handle.send_command(f"vbox 2drp {vbox_id}")
    return drp_id_new

def batch_mix(list_drp_id, mix_time):
    list_feature = []
    with ThreadPoolExecutor(max_workers=32) as executor:
        for drp_id in list_drp_id:
            feature = executor.submit(single_mix, drp_id, mix_time)
            list_feature.append(feature)
    list_drp_id_new = []
    for feature in list_feature:  
        list_drp_id_new.append(feature.result())
    return list_drp_id_new
mix_func = batch_mix

In [10]:
def move_drps_to_points(drps: list, positions: list, path_temp_point = False, temp_point_dir: str = 'up'):
    if path_temp_point:
        move_drps_to_tmp_pos(drps, positions, temp_point_dir)
    tags = []
    for id, pos in zip(drps, positions):
        tag = api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Vertical.value}")
        tags.append(tag)
    af.wait_tag_all(tags)

def move_vbox_p(vbox_id, target_x, target_y, dir: MoveDirection, speed = 2):
    curr_x, curr_y = get_area_central_point(api_handle.send_command(f"vbox getbox {vbox_id}"))
    if (MoveDirection.Horizontal.value == dir):
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {target_x} {curr_y} {speed}")
        )
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {target_x} {target_y} {speed}")
        )
    else:
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {curr_x} {target_y}")
        )
        api_handle.wait_tag( \
        api_handle.send_command(f"vbox move {vbox_id} {target_x} {target_y}")
        )

def move_shift_vbox(vbox_id, x_shift, y_shift, speed = 4):
    curr_x, curr_y = get_area_central_point(api_handle.send_command(f"vbox getbox {vbox_id}"))
    return api_handle.send_command(f"vbox move {vbox_id} {curr_x + x_shift} {curr_y + y_shift} {speed}")

In [11]:
split_volume = 1.67
orig_volume = split_volume * 9
orig_length = ONE_UL_LEN * orig_volume**0.5
split_length = ONE_UL_LEN * split_volume**0.5

matrix = af.get_mag_info(8, 4)
speed_drp = 0.25
speed_vbox = 2
matrix.extend([[853, 601], [853, 530], [853, 459], [853, 388], [853, 317], [853, 246], [853, 175], [853, 104]])

函数定义

In [12]:
def vbox_move_shift(vbox_id, x_distance, y_distance, speed):
    x1, y1, x2, y2 = api_handle.send_command("vbox getbox {}".format(vbox_id))
    x_destination = (x1 + x2)/2 + x_distance
    y_destination = (y1 + y2)/2 + y_distance
    tag = api_handle.send_command(f"vbox move {vbox_id} {x_destination} {y_destination} {speed}")
    return tag

def extract_droplet_v2(vbox_id, direction, orig_volume, split_volume, split_number, list_split_position, extract_time = 3):
    x1, y1, x2, y2 = api_handle.send_command("vbox getbox {}".format(vbox_id))
    api_handle.send_command(f"vbox remove {vbox_id}")

    list_vbox_id_saved = []
    step = 0
    while step < split_number:
        area = orig_volume - step * split_volume
        length_new = ONE_UL_LEN*(area-split_volume)**0.5
        split_length = ONE_UL_LEN * split_volume**0.5
        speed = 2
        sx_left = (x1+x2)*0.5 - length_new*0.5
        sx_right = (x1+x2)*0.5 + length_new*0.5
        sy_down = (y1+y2)*0.5 - length_new*0.5
        sy_up = (y1+y2)*0.5 + length_new*0.5
        # bottom_vbox
        if direction == 'down':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_right,sy_down+length_new*0.1))
        elif direction == 'up':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_up-length_new*0.1,sx_right,sy_up))
        elif direction == 'left':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-length_new*0.1,sy_down,sx_right,sy_up))
        elif direction == 'right':
            bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_left+length_new*0.1,sy_up))            
        # neck_vbox
        if direction == 'down':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/4,sy_up-split_length*0.5,(x1+x2)*0.5+split_length/4,sy_up+split_length*1.5))
        elif direction == 'up':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/4,sy_down-split_length*1.5,(x1+x2)*0.5+split_length/4,sy_down+split_length*0.5))
        elif direction == 'left':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*1.5,(y1+y2)/2-split_length/4,sx_left+split_length*0.5,(y1+y2)/2+split_length/4))
        elif direction == 'right':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-split_length*0.5,(y1+y2)/2-split_length/4,sx_right+split_length*1.5,(y1+y2)/2+split_length/4))
        time.sleep(1)
        
        # vbox_saved
        if direction == 'down':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/2,sy_up+split_length*1,(x1+x2)*0.5+split_length/2,sy_up+split_length*2))
        elif direction == 'up':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/2,sy_down-split_length*2,(x1+x2)*0.5+split_length/2,sy_down-split_length))
        elif direction == 'left':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*2,(y1+y2)*0.5-split_length/2,sx_left-split_length,(y1+y2)*0.5+split_length/2))
        elif direction == 'right':
            vbox_id_saved = api_handle.send_command("vbox add {} {} {} {}".format(sx_right+split_length,(y1+y2)*0.5-split_length/2,sx_right+split_length*2,(y1+y2)*0.5+split_length/2))
        list_vbox_id_saved.append(vbox_id_saved)
        time.sleep(extract_time)
        api_handle.send_command(f"vbox remove {neck_vbox_id}")
        api_handle.send_command(f"vbox remove {bottom_vbox_id}")
        
        # second_neck_vbox
        if direction == 'down':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/16,sy_up-split_length*0.5,(x1+x2)*0.5+split_length/16,sy_up+split_length*1))
        elif direction == 'up':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length/16,sy_down-split_length*1,(x1+x2)*0.5+split_length/16,sy_down+split_length*0.5))
        elif direction == 'left':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*1,(y1+y2)/2-split_length/16,sx_right-split_length*0.5,(y1+y2)/2+split_length/16))
        elif direction == 'right':
            neck_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-split_length*0.5,(y1+y2)/2-split_length/16,sx_right+split_length*1,(y1+y2)/2+split_length/16))
            
        bottom_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_right,sy_up))
        time.sleep(2)
        api_handle.send_command(f"vbox remove {neck_vbox_id}")
        
        # withdraw_vbox
        if direction == 'down':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length*0.5,sy_up-split_length*0.5,(x1+x2)*0.5+split_length*0.5,sy_up+split_length*0.5))
        elif direction == 'up':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)*0.5-split_length*0.5,sy_down-split_length*0.5,(x1+x2)*0.5+split_length*0.5,sy_down+split_length*0.5))
        elif direction == 'left':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left-split_length*0.5,(y1+y2)/2-split_length*0.5,sx_left+split_length*0.5,(y1+y2)/2+split_length*0.5))
        elif direction == 'right':
            withdraw_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_right-split_length/2,(y1+y2)/2-split_length*0.5,sx_right+split_length/2,(y1+y2)/2+split_length*0.5))
        time.sleep(0.5)
        api_handle.send_command(f"vbox remove {withdraw_vbox_id}")
        
        # move
        tag = None
        if direction == 'up':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[step][0]} {list_split_position[step][1]} {speed}")
        elif direction == 'down':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[split_number-step-1][0]} {list_split_position[split_number-step-1][1]} {speed}")
        elif direction == 'left':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[step][0]} {list_split_position[step][1]} {speed}")
        elif direction == 'right':
            tag = api_handle.send_command(f"vbox move {vbox_id_saved} {list_split_position[split_number-step-1][0]} {list_split_position[split_number-step-1][1]} {speed}")
            
        time.sleep(5)
        api_handle.send_command(f"vbox remove {bottom_vbox_id}")
        if step == split_number-1:
            rest_vbox_id = api_handle.send_command("vbox add {} {} {} {}".format(sx_left,sy_down,sx_right,sy_up))
            api_handle.wait_tag(tag)
        step += 1
        
    if direction == 'up' or direction == 'left':
        return list_vbox_id_saved, rest_vbox_id
    else:
        return list_vbox_id_saved[::-1], rest_vbox_id


In [13]:
def even_split(vbox_id, move_speed):
    x1, y1, x2, y2 = api_handle.send_command(f"vbox getbox {vbox_id}")
    api_handle.send_command(f"vbox remove {vbox_id}")
    length_vbox_end = (x2-x1)/2**0.5
    bar_height = 2
    vbox1 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox2 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox3 = api_handle.send_command("vbox add {} {} {} {}".format(x1,(y1+y2)/2-bar_height/2,x2,(y1+y2)/2+bar_height/2))
    time.sleep(1)
    tags = []
    tag = move_shift_vbox(vbox1, -(x2-x1)*0.75, 0, move_speed)
    tags.append(tag)
    tag = move_shift_vbox(vbox2, (x2-x1)*0.75, 0, move_speed)
    tags.append
    af.wait_tag_all(tags)
    time.sleep(1)
    api_handle.send_command(f"vbox remove {vbox3}") 
    return vbox1, vbox2

def batch_even_split(list_vbox_id, move_speed):
    list_feature = []
    with ThreadPoolExecutor(max_workers=128) as executor:
        for vbox_id in list_vbox_id:
            feature = executor.submit(even_split, vbox_id, move_speed)
            list_feature.append(feature)
    list_vbox_id_new = []
    for feature in list_feature:  
        list_vbox_id_new.append(feature.result())
    return list_vbox_id_new

def even_split_2(drp_id, move_speed):
    x1, y1, x2, y2 = api_handle.send_command(f"getbox {drp_id}")
    api_handle.send_command(f"set_virtual {drp_id} 1")
    length_vbox_end = (x2-x1)/2**0.5
    bar_height = 2
    vbox1 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox2 = api_handle.send_command("vbox add {} {} {} {}".format((x1+x2)/2-length_vbox_end/2,(y1+y2)/2-length_vbox_end/2,(x1+x2)/2+length_vbox_end/2,(y1+y2)/2+length_vbox_end/2))
    vbox3 = api_handle.send_command("vbox add {} {} {} {}".format(x1,(y1+y2)/2-bar_height/2,x2,(y1+y2)/2+bar_height/2))
    time.sleep(1)
    tags = []
    tag = move_shift_vbox(vbox1, -(x2-x1)*0.75, 0, move_speed)
    tags.append(tag)
    tag = move_shift_vbox(vbox2, (x2-x1)*0.75, 0, move_speed)
    tags.append
    af.wait_tag_all(tags)
    time.sleep(1)
    api_handle.send_command(f"vbox remove {vbox3}")
    id1 = api_handle.send_command(f"vbox 2drp {vbox1}")
    id2 = api_handle.send_command(f"vbox 2drp {vbox2}")
    return id1, id2

def batch_even_split_2(list_drp_id, move_speed):
    list_feature = []
    with ThreadPoolExecutor(max_workers=128) as executor:
        for drp_id in list_drp_id:
            feature = executor.submit(even_split_2, drp_id, move_speed)
            list_feature.append(feature)
    list_drp_id_new = []
    for feature in list_feature:
        list_drp_id_new.append(feature.result())
    return list_drp_id_new

In [14]:
def garbage_drps(drps: list, move_shift_x = 300, mag_up: bool = True):
    def garbage_func():
        if mag_up:
            api_handle.send_command("mag up")
            time.sleep(6)
        # af.move_shift_all(drps, -35, 0, True)
        # time.sleep(1)
        # af.move_shift_all(drps, 0, 35, True)
        # time.sleep(1)
        # af.move_shift_all(drps, 70, -70, True)
        # time.sleep(2)
        drps2 = mix_func(drps, 10)
        af.move_shift_all(drps2, 35, -35, True)
        time.sleep(2)
        
        if move_shift_x > 0:
            af.move_shift_all(drps2, move_shift_x, 0, True)
        # api_handle.send_command("mag down")
        
        y1 = get_droplet_central_point(drps2[0])[1]
        garbage_pos_x, garbage_pos_y = get_hole_pos_by_pipe_id(12)[0], 62
        def move_to_garbage_pos(id):
            tag = api_handle.send_command(f"move_p {id} {garbage_pos_x} {y1} {MoveDirection.Vertical.value}")
            api_handle.wait_tag(tag)
            tag = api_handle.send_command(f"move_p {id} {garbage_pos_x} {garbage_pos_y} {MoveDirection.Vertical.value}")
            api_handle.wait_tag(tag)
            api_handle.send_command(f"set_virtual {id} 0")
            
        x_st = get_droplet_central_point(drps2[-1])[0]
        for i in range(len(drps2) // 6):
            st = -(i+1)*6
            ed = -i*6 if i != 0 else None
            group = drps2[st:ed]
            for j, id in enumerate(group):
                t = threading.Thread(target = move_to_garbage_pos, args = (id, ))
                t.start()
            while(get_droplet_central_point(group[-1])[0] < x_st + 30):
                time.sleep(1)
            if (i != len(drps2) // 6 - 1):
                af.move_shift_all(drps2[:len(drps2) - (i + 1) * 6], 70, 0, True)
                     
    t = threading.Thread(target=garbage_func)
    t.start()
    return t

In [15]:
split_volume = 1
drp_num = 6

In [16]:
speed = 1.2

DNA片段化

5×Reaction Buffer & Transposome Mix V50

In [ ]:
# 5×Reaction Buffer + Transposome Mix V50 
# 28
drp_vol_reaction_buffer = 0.8
orig_volume = drp_vol_reaction_buffer * drp_num * 2 + 0.5
pos1_x = get_hole_pos_by_pipe_id(28)[0]
pos1 = [pos1_x, 110]
mags = get_mags(1, 2, 6, 3, arrange = 0)
list_split_position = mags
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
vbox_list_1, unused_1 = extract_droplet_v2(vbox_id1, 'down', orig_volume, drp_vol_reaction_buffer, drp_num * 2, list_split_position, extract_time=4)

In [ ]:
drps_reaction_buffer = []
for vbox_id in vbox_list_1:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    drps_reaction_buffer.append(drp_id)
    api_handle.send_command(f"set_speed {drp_id} {speed}")

基因组DNA

In [ ]:
# 基因组DNA
# 22
drp_vol_genome = 1.8
orig_volume = drp_vol_genome * 2
mags = get_mags(1, 0, 6, 1, arrange = 1)
pos1 = [get_hole_pos_by_pipe_id(30)[0] + 25, get_hole_pos_by_pipe_id(30)[1] + 10]
list_split_position1 = [[pos[0], pos[1] - 70] for pos in mags[:2]]
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

pos2 = [get_hole_pos_by_pipe_id(32)[0] + 25, get_hole_pos_by_pipe_id(32)[1]]
list_split_position2 = [[pos[0], pos[1] - 30] for pos in mags[2:4]]
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

pos3 = [get_hole_pos_by_pipe_id(34)[0] + 25, get_hole_pos_by_pipe_id(34)[1]]
list_split_position3 = [[pos[0], pos[1] - 30] for pos in mags[4:6]]
vbox_id3 = api_handle.send_command(f"vbox add {list_to_string(pos3)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

pos4 = [get_hole_pos_by_pipe_id(36)[0] + 25, get_hole_pos_by_pipe_id(36)[1]]
list_split_position4 = [[pos[0], pos[1] - 30] for pos in mags[6:8]]
vbox_id4 = api_handle.send_command(f"vbox add {list_to_string(pos4)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

pos5 = [get_hole_pos_by_pipe_id(38)[0] + 25, get_hole_pos_by_pipe_id(38)[1]]
list_split_position5 = [[pos[0], pos[1] - 30] for pos in mags[8:10]]
vbox_id5 = api_handle.send_command(f"vbox add {list_to_string(pos5)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

pos6 = [get_hole_pos_by_pipe_id(40)[0] + 25, get_hole_pos_by_pipe_id(40)[1]]
list_split_position6 = [[pos[0], pos[1] - 30] for pos in mags[10:12]]
vbox_id6 = api_handle.send_command(f"vbox add {list_to_string(pos6)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'right', orig_volume, drp_vol_genome, 2, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'right', orig_volume, drp_vol_genome, 2, list_split_position2)
    f3 = executor.submit(extract_droplet_v2, vbox_id3, 'right', orig_volume, drp_vol_genome, 2, list_split_position3)
    f4 = executor.submit(extract_droplet_v2, vbox_id4, 'right', orig_volume, drp_vol_genome, 2, list_split_position4)
    f5 = executor.submit(extract_droplet_v2, vbox_id5, 'right', orig_volume, drp_vol_genome, 2, list_split_position5)
    f6 = executor.submit(extract_droplet_v2, vbox_id6, 'right', orig_volume, drp_vol_genome, 2, list_split_position6)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
    vbox_list_3, unused_3 = f3.result()
    vbox_list_4, unused_4 = f4.result()
    vbox_list_5, unused_5 = f5.result()
    vbox_list_6, unused_6 = f6.result()

In [ ]:
drps_genome = []
for vbox_id in vbox_list_1 + vbox_list_2 + vbox_list_3 + vbox_list_4 + vbox_list_5 + vbox_list_6:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    drps_genome.append(drp_id)
    api_handle.send_command(f"set_speed {drp_id} {speed}")

In [ ]:
af.move_shift_all(drps_genome, 140, 0, wait = True)

In [ ]:
row_to_column_sort(drps_genome, rows = 6, cols = 2)
af.merge_all(row_to_column_sort(drps_genome, rows = 6, cols = 2), drps_reaction_buffer, wait = True)

In [ ]:
drps_sample = drps_reaction_buffer
drps_sample = mix_func(drps_sample, 10)

In [ ]:
# 片段化产物移动到加热区
# 660, 775
tags = []
for i, drp_id in enumerate(drps_sample):
    if i < 6:
        tx = 660
    else:
        tx = 775
    tags.append(move_to_x(drp_id, tx))
af.wait_tag_all(tags)

In [ ]:
drps_sample = mix_func(drps_sample, 10)

终止转座

加入 Terminate solution

In [ ]:
# 加入 Terminate solution
# 54 56号孔位
# Terminate
drp_vol_terminate = 0.5
orig_volume = drp_vol_terminate * drp_num
# 54
pos1_x = get_hole_pos_by_pipe_id(54)[0]
pos1 = [pos1_x, 600]
list_split_position1 = [[pos1_x, y] for _, y in mag_centers[1:7]]
# 56
pos2_x = get_hole_pos_by_pipe_id(56)[0]
pos2 = [pos2_x, 600]
list_split_position2 = [[pos2_x, y] for _, y in mag_centers[1:7]]

In [ ]:
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'up', orig_volume, drp_vol_terminate, drp_num, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, drp_vol_terminate, drp_num, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()

tag1 = api_handle.send_command(f"vbox move {unused_1} {list_to_string(get_hole_pos_by_pipe_id(0))} 4")
tag2 = api_handle.send_command(f"vbox move {unused_2} {list_to_string(get_hole_pos_by_pipe_id(0))} 4")
af.wait_tag_all([tag1, tag2])
api_handle.send_command(f"vbox remove {unused_1}")
api_handle.send_command(f"vbox remove {unused_2}")

terminates = []
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    terminates.append(drp_id)

In [ ]:
af.merge_all(terminates, drps_sample, wait = True)

In [ ]:
tmp_drps = drps_sample[3:6] + drps_sample[9:12]
af.move_shift_all(tmp_drps, 0, 20, True)

In [ ]:
drps_sample = mix_func(drps_sample, 10)

In [ ]:
af.move_shift_all(drps_sample[3:6] + drps_sample[9:12], 0, -20, True)

In [ ]:
res = batch_even_split_2(drps_sample, 2)

In [ ]:
drps_frags = [None] * 24
for i, pair in enumerate(res):
    if i < 6:
        drps_frags[i] = pair[0]
        drps_frags[i + 6] = pair[1]
    else:
        drps_frags[i + 6] = pair[0]
        drps_frags[i + 12] = pair[1]

In [ ]:
# drps_frags = mix_func(drps_frags, 20)

片段化产物纯化-Ampure磁珠

In [ ]:
# Ampure磁珠
drp_vol_ampure = 1.2
orig_volume = drp_vol_ampure * drp_num + 2
mags = get_mags(1, 0, 6, 0, arrange = 0)
# 26
pos1_x = get_hole_pos_by_pipe_id(26)[0]
pos1 = [pos1_x, 115]
list_split_position1 = [[pos1_x, y - 30] for _, y in mags]
# 24
pos2_x = get_hole_pos_by_pipe_id(24)[0]
pos2 = [pos2_x, 115]
list_split_position2 = [[pos2_x, y - 30] for _, y in mags]
# 44
pos3_x = get_hole_pos_by_pipe_id(44)[0]
pos3 = [pos3_x, 600]
list_split_position3 = [[pos3_x, y - 30] for _, y in mags]
# 46
pos4_x = get_hole_pos_by_pipe_id(46)[0]
pos4 = [pos4_x, 600]
list_split_position4 = [[pos4_x, y - 30] for _, y in mags]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id3 = api_handle.send_command(f"vbox add {list_to_string(pos3)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id4 = api_handle.send_command(f"vbox add {list_to_string(pos4)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'down', orig_volume, drp_vol_ampure, drp_num, list_split_position1, 4)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'down', orig_volume, drp_vol_ampure, drp_num, list_split_position2, 4)
    f3 = executor.submit(extract_droplet_v2, vbox_id3, 'up', orig_volume, drp_vol_ampure, drp_num, list_split_position3, 4)
    f4 = executor.submit(extract_droplet_v2, vbox_id4, 'up', orig_volume, drp_vol_ampure, drp_num, list_split_position4, 4)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
    vbox_list_3, unused_3 = f3.result()
    vbox_list_4, unused_4 = f4.result()

In [ ]:
ampures = []
for vbox_id in vbox_list_3 + vbox_list_1 + vbox_list_4 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {1}")
    ampures.append(drp_id)
print(ampures)

In [ ]:
t1 = threading.Thread(target = move_drps_to_tmp_pos, args = (ampures, get_mags(1, 0, 6, 3), 'up'))
t2 = threading.Thread(target = move_drps_to_points, args = (drps_frags, get_mags(1, 0, 6, 3), False))
t1.start()
t2.start()

In [ ]:
# 多余磁珠排废
t_x = get_hole_pos_by_pipe_id(12)[0]
t_y = 40
t1 = threading.Thread(target = move_vbox_p, args = (unused_1, t_x, t_y, MoveDirection.Horizontal.value))
t2 = threading.Thread(target = move_vbox_p, args = (unused_2, t_x, t_y, MoveDirection.Horizontal.value))
t1.start()
t2.start()

t_x, t_y = 984, 615
t1 = threading.Thread(target = move_vbox_p, args = (unused_3, t_x, t_y, MoveDirection.Vertical.value))
t2 = threading.Thread(target = move_vbox_p, args = (unused_4, t_x, t_y, MoveDirection.Vertical.value))
t1.start()
t2.start()

In [ ]:
# ampures向样本merge
af.merge_all(ampures, drps_frags, wait = True)

In [ ]:
# mix
drps_frags = mix_func(drps_frags, 10)

In [ ]:
# 已使用16, 14, 56, 58号孔位
# 移上清
for id in drps_frags:
    api_handle.send_command(f"set_speed {id} 0.5")

In [ ]:
mags = get_mags(1, 0, 6, 3,arrange=0)
drps_frags = []
for pos in mags:
    vbox = api_handle.send_command(f"vbox add {list_to_string(pos)} {ONE_UL_LEN * math.sqrt(3)}")
    drp_id = api_handle.send_command(f"vbox 2drp {vbox}")
    drps_frags.append(drp_id)

In [ ]:
t = garbage_drps(drps_frags)
t.join()

片段化产物纯化-wash-1

In [ ]:
# wash，共2次
# 目前已使用58, 56, 14 ,16, 54, 52号孔位
# wash buffer 从8,6,4号孔位进样，一个孔位进16ul，每颗液滴2ul，单孔位液体分成8颗液滴，3个孔位共计24颗液滴
washing_buffer_list = []
interval2 = 60

In [ ]:
drp_vol_washing_buffer = 1.5
orig_volume = drp_vol_washing_buffer * 8 + 1
# pipe: 8
pos1 = [945, get_hole_pos_by_pipe_id(8)[1]]
pos1_x = pos1[0]
list_split_position1 = [[910 - interval2 * i, pos1[1]] for i in range(8)]
list_split_position1.reverse()
# pipe: 6
pos2 = [945, get_hole_pos_by_pipe_id(6)[1]]
pos2_x = pos2[0]
list_split_position2 = [[910 - interval2 * i, pos2[1]] for i in range(8)]
list_split_position2.reverse()
# pipe: 4
pos3 = [945, get_hole_pos_by_pipe_id(4)[1]]
pos3_x = pos3[0]
list_split_position3 = [[910 - interval2 * i, pos3[1]] for i in range(8)]
list_split_position3.reverse()

In [ ]:
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id3 = api_handle.send_command(f"vbox add {list_to_string(pos3)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position1, extract_time = 3)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position2, extract_time = 3)
    f3 = executor.submit(extract_droplet_v2, vbox_id3, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position3, extract_time = 3)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
    vbox_list_3, unused_3 = f3.result()
for vbox_id in vbox_list_1 + vbox_list_2 + vbox_list_3:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    washing_buffer_list.append(drp_id)
print(washing_buffer_list)

In [ ]:
for unused in [unused_1, unused_2, unused_3]:
    api_handle.send_command(f"vbox move {unused} {list_to_string(get_hole_pos_by_pipe_id(12))} 4")

In [ ]:
# 统一对齐
tmp_poses = get_mags(1, 0, 6, 3, arrange = 1)
tags = []
for drp_id, pos in zip(washing_buffer_list, tmp_poses):
    tags.append(move_to_y(drp_id, pos[1] - y_offset))
af.wait_tag_all(tags)

In [ ]:
tags = [api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Horizontal.value}") for id, pos in zip(washing_buffer_list, tmp_poses)]
af.wait_tag_all(tags)

In [ ]:
washing_buffer_list = row_to_column_sort(washing_buffer_list)

In [ ]:
washing_buffer_list = mix_func(washing_buffer_list, 10)
[api_handle.send_command(f"set_speed {id} {speed}") for id in washing_buffer_list]

In [ ]:
# 移上清
drps = washing_buffer_list
t = garbage_drps(drps)

片段化产物纯化-wash-2

In [ ]:
washing_buffer_list = []
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id3 = api_handle.send_command(f"vbox add {list_to_string(pos3)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position2)
    f3 = executor.submit(extract_droplet_v2, vbox_id3, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position3)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
    vbox_list_3, unused_3 = f3.result()
for vbox_id in vbox_list_1 + vbox_list_2 + vbox_list_3:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {1.3}")
    washing_buffer_list.append(drp_id)
print(washing_buffer_list)

In [ ]:
for unused in [unused_1, unused_2, unused_3]:
    api_handle.send_command(f"vbox move {unused} {list_to_string(get_hole_pos_by_pipe_id(12))} 4")

In [ ]:
# 统一对齐
tmp_poses = get_mags(1, 0, 6, 3, arrange = 1)
tags = []
for drp_id, pos in zip(washing_buffer_list, tmp_poses):
    tags.append(move_to_y(drp_id, pos[1] - y_offset))
af.wait_tag_all(tags)

tags = [api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Horizontal.value}") for id, pos in zip(washing_buffer_list, tmp_poses)]
af.wait_tag_all(tags)

In [ ]:
washing_buffer_list = row_to_column_sort(washing_buffer_list)

In [ ]:
washing_buffer_list = mix_func(washing_buffer_list, 10)
[api_handle.send_command(f"set_speed {id} {1.3}") for id in washing_buffer_list]

In [ ]:
# 移上清
drps = washing_buffer_list
t = garbage_drps(drps)
t.join()

片段化产物纯化-ddH20

In [ ]:
# 目前已使用58, 56, 14 ,16, 54, 52, 8, 6, 4号孔位
# ddH20从20, 50号孔位进样
# 因现在一颗大液滴被分成了2颗小液滴，故进样时每颗液滴要进预设体积的一半，即1/2 = 0.5ul
# 每个孔位进12颗，每颗0.5ul，每个孔位进样12*0.5 = 6ul
interval_y = 45
ddH20_list = []
drp_vol_ddH20 = 0.5
drp_num_ddH20 = 12
orig_volume = drp_vol_ddH20 * drp_num_ddH20 + 1
# pipe: 20
pos1_x = get_hole_pos_by_pipe_id(20)[0]
pos1 = [pos1_x, get_hole_pos_by_pipe_id(20)[1] + 50]
list_split_position1 = [[pos1_x, pos1[1] + interval_y * i] for i in range(12)]
# pipe: 50
pos2_x = get_hole_pos_by_pipe_id(50)[0]
pos2 = [pos2_x, get_hole_pos_by_pipe_id(50)[1] - 50]
list_split_position2 = [[pos2_x, pos2[1] - interval_y * i] for i in range(12)][::-1]

In [ ]:
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'down', orig_volume, drp_vol_ddH20, drp_num_ddH20, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, drp_vol_ddH20, drp_num_ddH20, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    ddH20_list.append(drp_id)

In [ ]:
move_shift_vbox(unused_1, 0, -50, 2)
move_shift_vbox(unused_2, 0, 50, 2)

In [ ]:
mags = get_mags(1, 0, 6, 3, arrange=0)
ddH20_list_sorted = ddH20_list[-len(ddH20_list)//2:] + ddH20_list[:len(ddH20_list)//2]
num_of_group = len(ddH20_list_sorted) // 4
# 第一组和最后一组向两边移动
af.move_shift_all(ddH20_list_sorted[:num_of_group], -30, 0, False)
af.move_shift_all(ddH20_list_sorted[-num_of_group:], 30, 0, True)
time.sleep(1)
# 对齐
tags = []
for drp_id, pos in zip(ddH20_list_sorted, mags):
    tags.append(move_to_y(drp_id, pos[1] - y_offset))
af.wait_tag_all(tags)
# 移动至磁控点
tags = []
for drp_id, pos in zip(ddH20_list_sorted, mags):
    tags.append(move_to_x(drp_id, pos[0]))
af.wait_tag_all(tags)
tags = []
for drp_id, pos in zip(ddH20_list_sorted, mags):
    tags.append(move_to_y(drp_id, pos[1]))
af.wait_tag_all(tags)

In [ ]:
ddH20_list = mix_func(ddH20_list_sorted, 10)
[api_handle.send_command(f"set_speed {id} {speed}") for id in ddH20_list]

片段化产物纯化-ddH20移动到暂存位置

In [ ]:
# 先将后面两排移过来
length = 12
drps = ddH20_list[:length]

tags = []
for id, (x, y) in zip(drps, mags[-length:]):
    tags.append(api_handle.send_command(f"move_p {id} {x} {y - 35} {MoveDirection.Vertical.value}"))
af.wait_tag_all(tags)

af.merge_all(ddH20_list[:length], ddH20_list[-length:], True)
ddH20_list = ddH20_list[-length:]

In [ ]:
ddH20_list = mix_func(ddH20_list, 10)

In [ ]:
api_handle.send_command("mag up")
time.sleep(5)
# 斜对角移动，让磁珠充分吸附在磁控点上
# 右
af.move_shift_all(ddH20_list, 30, 0, True)
# 下
af.move_shift_all(ddH20_list, 0, 30, True)
time.sleep(1)
# 左上
af.move_shift_all(ddH20_list, -60, -65)
# 将有效液体移至左边
move_drps_to_points(ddH20_list, [[x, y - 35] for x, y in mags[:12]], False)

片段化产物纯化-加水重悬磁珠并排废

In [ ]:
# 为了移除磁珠，需再加入水使磁珠重悬
# 50, 20号孔位各进6颗1ul液滴
# 并行处理50号和20号孔位的液滴提取
orig_volume = 7
# pipe: 50
pos1_x = get_hole_pos_by_pipe_id(50)[0]
pos1 = [pos1_x, get_hole_pos_by_pipe_id(50)[1] - 40]
list_split_position1 = [[pos1_x, y] for _, y in get_mags(1, 0, 6, 0, arrange=1)]
# pipe: 20
pos2_x = get_hole_pos_by_pipe_id(20)[0]
pos2 = [pos2_x, get_hole_pos_by_pipe_id(20)[1] + 40]
list_split_position2 = [[pos2_x, y] for _, y in get_mags(1, 0, 6, 0, arrange=1)]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'up', orig_volume, 1, 6, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'down', orig_volume, 1, 6, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
tmp_H20_list = []
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    tmp_H20_list.append(drp_id)

In [ ]:
pos_pipe0_x, pos_pipe0_y = get_hole_pos_by_pipe_id(0)
pos_pipe12_x, pos_pipe12_y = get_hole_pos_by_pipe_id(12)

In [ ]:
t1 = threading.Thread(target = move_vbox_p, args = (unused_1, pos_pipe0_x, pos_pipe0_y, MoveDirection.Vertical.value))
t1.start()
t2 = threading.Thread(target = move_vbox_p, args = (unused_2, pos_pipe12_x, pos_pipe12_y, MoveDirection.Horizontal.value))
t2.start()

In [ ]:
mags = get_mags(1, 2, 6, 3, arrange=0)
move_drps_to_points(tmp_H20_list, mags, True, 'up')

In [ ]:
api_handle.send_command("mag down")
time.sleep(2)
# mix使其充分重悬
tmp_H20_list = mix_func(tmp_H20_list, 10)

In [ ]:
# garbabe beads
drps = tmp_H20_list
t = garbage_drps(drps, mag_up=False)
t.join()

In [ ]:
# 将片段化纯化产物(ddH20)移回磁控点
mags = get_mags(1, 2, 6, 3, arrange=0)
move_drps_to_points(ddH20_list, mags, path_temp_point=True, temp_point_dir='up')

PCR扩增-加入PCR

In [ ]:
# 加入12个sPCR
# 目前已使用58, 56, 54, 52, 50, 14 ,16, 20, 8, 6, 4号孔位
drps_pcr = []
# PCR从18号孔位进样，每次进6颗，每颗1.3 ul
pos_x, pos_y = get_hole_pos_by_pipe_id(18)[0], 120
drp_vol_pcr = 1.3
orig_volume = drp_vol_pcr * drp_num + 1.5
list_split_position = [[pos_x, y - 30] for _, y in mag_centers[1:7]]
vbox_id = api_handle.send_command(f"vbox add {pos_x} {pos_y} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
vbox_list, unused = extract_droplet_v2(vbox_id, 'down', orig_volume, drp_vol_pcr, 6, list_split_position)

In [ ]:
pos_pipe12_x, pos_pipe12_y = get_hole_pos_by_pipe_id(12)
t = threading.Thread(target = move_vbox_p, args = (unused, pos_pipe12_x, pos_pipe12_y, MoveDirection.Horizontal.value))
t.start()

In [ ]:
for vbox_id in vbox_list:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps_pcr.append(drp_id)

In [ ]:
move_drps_to_tmp_pos(drps_pcr, get_mags(1, 2, 6, 2, arrange=0), 'up')

In [ ]:
drps_pcr2 = []
# PCR从18号孔位进样，每次进6颗，每颗1.3 ul
pos_x, pos_y = get_hole_pos_by_pipe_id(18)[0], 120
drp_vol_pcr = 1.3
orig_volume = drp_vol_pcr * drp_num + 1.5
list_split_position = [[pos_x, y - 30] for _, y in mag_centers[1:7]]
vbox_id = api_handle.send_command(f"vbox add {pos_x} {pos_y} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
vbox_list, unused = extract_droplet_v2(vbox_id, 'down', orig_volume, drp_vol_pcr, 6, list_split_position)

pos_pipe12_x, pos_pipe12_y = get_hole_pos_by_pipe_id(12)
t = threading.Thread(target = move_vbox_p, args = (unused, pos_pipe12_x, pos_pipe12_y, MoveDirection.Horizontal.value))
t.start()

for vbox_id in vbox_list:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps_pcr2.append(drp_id)

drps_pcr.extend(drps_pcr2)
move_drps_to_tmp_pos(drps_pcr2, get_mags(1, 3, 6, 3, arrange=0), 'up')

In [ ]:
# PCR进样完毕，和片段化纯化产物合并
af.merge_all(drps_pcr, ddH20_list, wait = True)

PCR扩增-primer进样

In [ ]:
# 12个primer，一次进6颗，分两批进样
# 目前已使用58, 56, 54, 52, 50, 14 ,16, 18, 20, 8, 6, 4号孔位
# 2号孔位进样
# 第一批
drps_primer = []
pos_x, pos_y = 942, get_hole_pos_by_pipe_id(2)[1]
drp_vol_primer = 0.3
orig_volume = drp_vol_primer * drp_num + 0.5
list_split_position = [[420, y - 30] for _, y in mag_centers[1:7]]

In [ ]:
vbox_id = api_handle.send_command(f"vbox add {pos_x} {pos_y} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
vbox_list, unused = extract_droplet_v2(vbox_id, 'left', orig_volume, drp_vol_primer, 6, list_split_position)

In [ ]:
drps = []
for vbox_id in vbox_list:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps.append(drp_id)
move_drps_to_tmp_pos(drps, get_mags(1, 2, 6, 2, arrange=0), 'up')

In [ ]:
drps_primer.extend(drps)

In [ ]:
# 第二批
vbox_id = api_handle.send_command(f"vbox add {pos_x} {pos_y} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
vbox_list, unused = extract_droplet_v2(vbox_id, 'left', orig_volume, drp_vol_primer, 6, list_split_position)

drps = []
for vbox_id in vbox_list:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps.append(drp_id)
move_drps_to_tmp_pos(drps, get_mags(1, 3, 6, 3, arrange=0), 'up')

In [ ]:
drps_primer.extend(drps)

In [ ]:
# primer进样完毕，和PCR混匀
af.merge_all(drps_primer, ddH20_list, wait = True)

In [ ]:
ddH20_list = mix_func(ddH20_list, 10)

In [ ]:
x_shift = 686 - get_droplet_central_point(ddH20_list[0])[0]
af.move_shift_all(ddH20_list, x_shift, 0, wait = True)

In [ ]:
# 因体积过大需要拆分
drps_pcr_result_tmp = batch_even_split_2(ddH20_list, 2)

In [ ]:
drps_pcr_result = [None] * 24
for i, pair in enumerate(drps_pcr_result_tmp):
    if i < 6:
        drps_pcr_result[i] = pair[0]
        drps_pcr_result[i + 6] = pair[1]
    else:
        drps_pcr_result[i + 6] = pair[0]
        drps_pcr_result[i + 12] = pair[1]

In [ ]:
af.move_shift_all(drps_pcr_result[:12], -30, 0, wait = True)

In [ ]:
# 在加热区加热
# 30s 72℃, 30s 95℃
# 10s 95℃, 30s 55℃, 30s 72℃ (5-9 cycles)
# 5min 72℃

# def heater(target_temp, duration):
#     api_handle.send_command(f"heater skip {0}")
#     api_handle.send_command(f"heater skip {1}")
#     api_handle.send_command(f"heater add_cycle {0} {target_temp} {duration + 10} {0} {0}")
#     api_handle.send_command(f"heater add_cycle {1} {target_temp} {duration + 10} {0} {0}")
#     api_handle.send_command("heater start 0")
#     api_handle.send_command("heater start 1")
    
# def heater(parameters: list):
    

drps_pcr_result = mix_func(drps_pcr_result, 10)

In [ ]:
# 将分裂后的液滴移动到磁控点摆放规则
move_drps_to_points(drps_pcr_result, get_mags(1, 0, 6, 3, arrange=0))

PCR产物纯化-加入1st 磁珠

In [ ]:
# 加入磁珠 1st
# 目前已使用58, 56, 54, 52, 50, 14 ,16, 18, 20, 8, 6, 4, 2号孔位
# 使用52、54号孔位
drps_ampure_1st_1 = []
mags = mag_centers[1:7]
orig_volume = 7
# 52
pos1_x = get_hole_pos_by_pipe_id(52)[0]
pos1 = [pos1_x, 600]
list_split_position1 = [[pos1_x, y - 35] for _, y in mags]
# 54
pos2_x = get_hole_pos_by_pipe_id(54)[0]
pos2 = [pos2_x, 600]
list_split_position2 = [[pos2_x, y - 35] for _, y in mags]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'up', orig_volume, 1, 6, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, 1, 6, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps_ampure_1st_1.append(drp_id)

In [ ]:
# 下方液滴的排废孔在0号孔
# time.sleep(90000)
t_x, t_y = 1012, 640
t1 = threading.Thread(target = move_vbox_p, args = (unused_1, t_x, t_y, MoveDirection.Vertical.value))
t2 = threading.Thread(target = move_vbox_p, args = (unused_2, t_x, t_y, MoveDirection.Vertical.value))
t1.start()
t2.start()

In [ ]:
move_drps_to_points(drps_ampure_1st_1, get_mags(1, 0, 6, 1, arrange=0), True, 'up')

In [ ]:
first_half = drps_pcr_result[:12]
af.merge_all(drps_ampure_1st_1, first_half, True)

In [ ]:
def async_func1():
    global first_half
    first_half = mix_func(first_half, 200)
thread_mix = threading.Thread(target = async_func1)
thread_mix.start()

In [ ]:
drps_ampure_1st_2 = []
mags = mag_centers[1:7]
orig_volume = 7
# 52
pos1_x = get_hole_pos_by_pipe_id(52)[0]
pos1 = [pos1_x, 600]
list_split_position1 = [[pos1_x, y - 35] for _, y in mags]
# 54
pos2_x = get_hole_pos_by_pipe_id(54)[0]
pos2 = [pos2_x, 600]
list_split_position2 = [[pos2_x, y - 35] for _, y in mags]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'up', orig_volume, 1, 6, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, 1, 6, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps_ampure_1st_2.append(drp_id)

In [ ]:
# 下方液滴的排废孔在0号孔
t_x, t_y = 1012, 640
garbage_unused_t1 = threading.Thread(target = move_vbox_p, args = (unused_1, t_x, t_y, MoveDirection.Vertical.value))
garbage_unused_t2 = threading.Thread(target = move_vbox_p, args = (unused_2, t_x, t_y, MoveDirection.Vertical.value))
garbage_unused_t1.start()
garbage_unused_t2.start()

In [ ]:
move_drps_to_points(drps_ampure_1st_2, get_mags(1, 2, 6, 3, arrange=0), True, 'up')

In [ ]:
latter_half = drps_pcr_result[-12:]
af.merge_all(drps_ampure_1st_2, latter_half, True)

In [ ]:
latter_half = mix_func(latter_half, 20)
thread_mix.join()

In [ ]:
drps_ampure_1st = first_half + latter_half

In [ ]:
# 留上清液，去磁珠
api_handle.send_command("mag up")
time.sleep(5)
drps_ampure_1st = mix_func(drps_ampure_1st, 5)
time.sleep(1)
af.move_shift_all(drps_ampure_1st, 35, -35, wait = True)

In [ ]:
# 加 1ul 水
# TODO: 移除磁珠

In [ ]:
# 上清液归位
af.move_shift_all(drps_ampure_1st, -35, 35, wait = True)

PCR产物纯化-加入2nd 磁珠

In [ ]:
# 加入磁珠 2nd
# 目前已使用58, 56, 54, 52, 50, 14 ,16, 18, 20, 8, 6, 4, 2号孔位
# 使用52、54号孔位
drps_ampure_2nd = []
mags = mag_centers[1:7]
orig_volume = 3.5
# 52
pos1_x = get_hole_pos_by_pipe_id(52)[0]
pos1 = [pos1_x, 600]
list_split_position1 = [[pos1_x, y - 35] for _, y in mags]
# 54
pos2_x = get_hole_pos_by_pipe_id(54)[0]
pos2 = [pos2_x, 600]
list_split_position2 = [[pos2_x, y - 35] for _, y in mags]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'up', orig_volume, 0.5, 6, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, 0.5, 6, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps_ampure_2nd.append(drp_id)

In [ ]:
# 下方液滴的排废孔在0号孔
t_x, t_y = 1012, 640
t1 = threading.Thread(target = move_vbox_p, args = (unused_1, t_x, t_y, MoveDirection.Vertical.value))
t2 = threading.Thread(target = move_vbox_p, args = (unused_2, t_x, t_y, MoveDirection.Vertical.value))
t1.start()
t2.start()

In [ ]:
move_drps_to_points(drps_ampure_2nd, get_mags(1, 0, 6, 1, arrange=0), True, 'up')

In [ ]:
first_half = drps_ampure_1st[:12]
af.merge_all(drps_ampure_2nd, first_half, True)

In [ ]:
def async_func_mix_ampure2nd_first_half():
    global first_half
    first_half = mix_func(first_half, 30)
    [api_handle.send_command(f"set_speed {id} {speed}") for id in first_half]
t1 = threading.Thread(target=async_func_mix_ampure2nd_first_half)
t1.start()

In [ ]:
# 加入磁珠 2nd
# 目前已使用58, 56, 54, 52, 50, 14 ,16, 18, 20, 8, 6, 4, 2号孔位
# 使用52、54号孔位
drps_ampure_2nd = []
mags = mag_centers[1:7]
orig_volume = 3.5
# 52
pos1_x = get_hole_pos_by_pipe_id(52)[0]
pos1 = [pos1_x, 600]
list_split_position1 = [[pos1_x, y - 35] for _, y in mags]
# 54
pos2_x = get_hole_pos_by_pipe_id(54)[0]
pos2 = [pos2_x, 600]
list_split_position2 = [[pos2_x, y - 35] for _, y in mags]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'up', orig_volume, 0.5, 6, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, 0.5, 6, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    drps_ampure_2nd.append(drp_id)

In [ ]:
# 下方液滴的排废孔在0号孔
t_x, t_y = 1012, 640
garbage_unused_t1 = threading.Thread(target = move_vbox_p, args = (unused_1, t_x, t_y, MoveDirection.Vertical.value))
garbage_unused_t2 = threading.Thread(target = move_vbox_p, args = (unused_2, t_x, t_y, MoveDirection.Vertical.value))
garbage_unused_t1.start()
garbage_unused_t2.start()

In [ ]:
move_drps_to_points(drps_ampure_2nd, get_mags(1, 2, 6, 3, arrange=0), True, 'up')

In [ ]:
latter_half = drps_ampure_1st[-12:]
af.merge_all(drps_ampure_2nd, latter_half, True)

In [ ]:
latter_half = mix_func(latter_half, 2)
t1.join()

In [ ]:
# 移上清
api_handle.send_command("mag up")
time.sleep(5)

In [ ]:
drps = first_half + latter_half
t = garbage_drps(drps)
t.join()

PCR产物纯化-wash-1

In [ ]:
washing_buffer_list = []
interval2 = 40
drp_vol_washing_buffer = 1
orig_volume = drp_vol_washing_buffer * 8 + 1
x = 400
# pipe: 8
pos1 = [945, get_hole_pos_by_pipe_id(8)[1]]
list_split_position1 = [None] * 8
for i in range(2):
    for j in range(4):
        list_split_position1[i*4+j] = [x + interval2 * j, 158 + 70 * i]

# pipe: 6
pos2 = [945, get_hole_pos_by_pipe_id(6)[1]]
list_split_position2 = [None] * 8
for i in range(2):
    for j in range(4):
        list_split_position2[i*4+j] = [x + interval2 * j, 158 + 140 + 70 * i]

# pipe: 4
pos3 = [945, get_hole_pos_by_pipe_id(4)[1]]
list_split_position3 = [None] * 8
for i in range(2):
    for j in range(4):
        list_split_position3[i*4+j] = [x + interval2 * j, 158 + 280 + 70 * i]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id3 = api_handle.send_command(f"vbox add {list_to_string(pos3)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor(max_workers = 16) as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position1, extract_time = 1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position2, extract_time = 1)
    f3 = executor.submit(extract_droplet_v2, vbox_id3, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position3, extract_time = 1)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
    vbox_list_3, unused_3 = f3.result()

In [ ]:
for vbox_id in vbox_list_1 + vbox_list_2 + vbox_list_3:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    washing_buffer_list.append(drp_id)
print(washing_buffer_list)

In [ ]:
for unused in [unused_1, unused_2, unused_3]:
    api_handle.send_command(f"vbox move {unused} {list_to_string(get_hole_pos_by_pipe_id(12))} 4")

In [ ]:
# 统一对齐
tmp_poses = get_mags(1, 0, 6, 3, arrange = 1)
tags = []
for drp_id, pos in zip(washing_buffer_list, tmp_poses):
    tags.append(move_to_y(drp_id, pos[1] - y_offset))
af.wait_tag_all(tags)
tags = [api_handle.send_command(f"move_p {id} {list_to_string(pos)} {MoveDirection.Horizontal.value}") for id, pos in zip(washing_buffer_list, tmp_poses)]
af.wait_tag_all(tags)

In [ ]:
washing_buffer_list_sorted = row_to_column_sort(washing_buffer_list)
washing_buffer_list = mix_func(washing_buffer_list_sorted, 10)

In [ ]:
# 移上清
drps = washing_buffer_list
t = garbage_drps(drps)
t.join()

PCR产物纯化-wash-2

In [ ]:
washing_buffer_list = []
interval2 = 40
drp_vol_washing_buffer = 1.5
orig_volume = drp_vol_washing_buffer * 8 + 1
x = 400
# pipe: 8
pos1 = [945, get_hole_pos_by_pipe_id(8)[1]]
list_split_position1 = [None] * 8
for i in range(2):
    for j in range(4):
        list_split_position1[i*4+j] = [x + interval2 * j, 158 + 70 * i]

# pipe: 6
pos2 = [945, get_hole_pos_by_pipe_id(6)[1]]
list_split_position2 = [None] * 8
for i in range(2):
    for j in range(4):
        list_split_position2[i*4+j] = [x + interval2 * j, 158 + 140 + 70 * i]

# pipe: 4
pos3 = [945, get_hole_pos_by_pipe_id(4)[1]]
list_split_position3 = [None] * 8
for i in range(2):
    for j in range(4):
        list_split_position3[i*4+j] = [x + interval2 * j, 158 + 280 + 70 * i]

vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id3 = api_handle.send_command(f"vbox add {list_to_string(pos3)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position1, extract_time = 6)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position2, extract_time = 6)
    f3 = executor.submit(extract_droplet_v2, vbox_id3, 'left', orig_volume, drp_vol_washing_buffer, 8, list_split_position3, extract_time = 6)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
    vbox_list_3, unused_3 = f3.result()
for vbox_id in vbox_list_1 + vbox_list_2 + vbox_list_3:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    washing_buffer_list.append(drp_id)
print(washing_buffer_list)

In [ ]:
washing_buffer_list = row_to_column_sort(washing_buffer_list)

In [ ]:
move_drps_to_points(washing_buffer_list, get_mags(1, 0, 6, 3, arrange=0), True, 'up')

In [ ]:
washing_buffer_list = mix_func(washing_buffer_list, 10)

In [ ]:
# 移上清
t = garbage_drps(washing_buffer_list)
t.join()

PCR产物纯化-ddH20

In [ ]:
interval_y = 45
ddH20_list = []
drp_vol_ddH20 = 0.5
drp_num_ddH20 = 12
orig_volume = drp_vol_ddH20 * drp_num_ddH20 + 1
# pipe: 20
pos1_x = get_hole_pos_by_pipe_id(20)[0]
pos1 = [pos1_x, get_hole_pos_by_pipe_id(20)[1] + 50]
list_split_position1 = [[pos1_x, pos1[1] + interval_y * i] for i in range(12)]
# pipe: 50
pos2_x = get_hole_pos_by_pipe_id(50)[0]
pos2 = [pos2_x, get_hole_pos_by_pipe_id(50)[1] - 50]
list_split_position2 = [[pos2_x, pos2[1] - interval_y * i] for i in range(12)][::-1]

In [ ]:
vbox_id1 = api_handle.send_command(f"vbox add {list_to_string(pos1)} {ONE_UL_LEN * math.sqrt(orig_volume)}")
vbox_id2 = api_handle.send_command(f"vbox add {list_to_string(pos2)} {ONE_UL_LEN * math.sqrt(orig_volume)}")

In [ ]:
with ThreadPoolExecutor() as executor:
    f1 = executor.submit(extract_droplet_v2, vbox_id1, 'down', orig_volume, drp_vol_ddH20, drp_num_ddH20, list_split_position1)
    f2 = executor.submit(extract_droplet_v2, vbox_id2, 'up', orig_volume, drp_vol_ddH20, drp_num_ddH20, list_split_position2)
    vbox_list_1, unused_1 = f1.result()
    vbox_list_2, unused_2 = f2.result()
for vbox_id in vbox_list_1 + vbox_list_2:
    drp_id = api_handle.send_command(f"vbox 2drp {vbox_id}")
    api_handle.send_command(f"set_speed {drp_id} {speed}")
    ddH20_list.append(drp_id)

In [ ]:
ddH20_list_sorted = ddH20_list[-len(ddH20_list)//2:] + ddH20_list[:len(ddH20_list)//2]
num_of_group = len(ddH20_list_sorted) // 4
mags = get_mags(1, 0, 6, 3, arrange=0)
# 第一组和最后一组向两边移动
af.move_shift_all(ddH20_list_sorted[:num_of_group], -30, 0, False)
af.move_shift_all(ddH20_list_sorted[-num_of_group:], 30, 0, True)
time.sleep(1)
# 对齐
tags = []
for drp_id, pos in zip(ddH20_list_sorted, mags):
    tags.append(move_to_y(drp_id, pos[1] - y_offset))
af.wait_tag_all(tags)
# 移动至磁控点
tags = []
for drp_id, pos in zip(ddH20_list_sorted, mags):
    tags.append(move_to_x(drp_id, pos[0]))
af.wait_tag_all(tags)
tags = []
for drp_id, pos in zip(ddH20_list_sorted, mags):
    tags.append(move_to_y(drp_id, pos[1]))
af.wait_tag_all(tags)

In [ ]:
ddH20_list_sorted = mix_func(ddH20_list_sorted, 30)

In [ ]:
api_handle.send_command("mag up")

In [ ]:
drps = ddH20_list_sorted

In [ ]:
af.move_shift_all(drps, -35, 0, True)
time.sleep(1)
af.move_shift_all(drps, 0, 35, True)
time.sleep(1)
af.move_shift_all(drps, 70, -70, True)
time.sleep(2)

In [ ]:
y1 = get_droplet_central_point(drps[0])[1]
garbage_pos_x, garbage_pos_y = get_hole_pos_by_pipe_id(12)[0], 62
def move_to_garbage_pos(id):
    garbage_pos_x, y1 = 455, 62
    tag = api_handle.send_command(f"move_p {id} {garbage_pos_x} {y1} {MoveDirection.Vertical.value}")
    api_handle.wait_tag(tag)
    tag = api_handle.send_command(f"move_p {id} {garbage_pos_x} {garbage_pos_y} {MoveDirection.Vertical.value}")
    api_handle.wait_tag(tag)
    api_handle.send_command(f"set_virtual {id} 0")
    
x_st = get_droplet_central_point(drps[-1])[0]
for i in range(len(drps) // 6):
    st = -(i+1)*6
    ed = -i*6 if i != 0 else None
    group = drps[st:ed]
    for j, id in enumerate(group):
        t = threading.Thread(target = move_to_garbage_pos, args = (id, ))
        t.start()
    while(get_droplet_central_point(group[-1])[0] < x_st + 30):
        time.sleep(1)
    if (i != len(drps) // 6 - 1):
        af.move_shift_all(drps[:len(drps) - (i + 1) * 6], 70, 0, True)

In [ ]:
# mags_tmp = get_mags(1, 0, 6, 3, arrange=0)
# drps_ampure_1st = []
# for pos in mags_tmp:
#     vbox = api_handle.send_command(f"vbox add {list_to_string(pos)} {ONE_UL_LEN * 1.4142}")
#     drps_ampure_1st.append(api_handle.send_command(f"vbox 2drp {vbox}"))    

In [ ]:
jsons = api_handle.list_droplets()
for obj in jsons:
    id = obj['id']
    # print(id)
    api_handle.send_command(f"set_virtual {id} 0")